# The convolution operation

The convolution operation between a 2D tensor $I$ (image) and e 2D tensor $K$ ("kernel") is usually defined in deep learning libraries as the matrix $S$ with entries
$$
S_{ij} = \sum_k \sum_l K_{i+k, j+l}\,I_{kl}
$$

__Note:__ technically the above is known as "cross-correlation", but the misnomer has stuck within the deep learning community (the proper "convolution" differing only by the signs in $K$'s indices, which are respectively $i-k$ and $j-l$ in the proper case).

Because a the notion of a "center position" (pixel) for the kernel is needed, kernels are taken to have odd dimensions. Moreover, square kernels are usually used.

Given an image matrix of size $N\times M$ and a kernel of size $F\times F$, the kernel has a "frame" around the center of size $(F-1)/2$ and the resulting output (if no padding is added back after the convolution operation and with unit stride) has size $(M-F+1)\times (N-F+1)$ (each dimension is subtracted twice the size of the kernel's frame). $F$ is said to be the **kernel's receptive field**.

Strides $S$ measures the step by which the center of the kernel is moved at each step across the image. $S=1$ correspond to the formula above, while for general (integer) values we have that $S - 1$ pixels are "skipped" at each step. In order for the kernel to perform a correct tiling of the image (covering all of it as it moves), the number of steps along all directions (starting at a corner, modulo the kernel's frame size) must be integer,
$$
\frac{W - F}{S} + 1 \in \mathbb{N}\,.
$$

__Note:__ the implementation of the convolution operation below is sequential and thus very inefficient. In fact, convolution is trivially parallelizable because the kernel acts of different patches of the image in a completely independent way.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
def convolve(image, kernel, stride=1):
    """
    Computes the convolution operation between an image
    and a kernel. The image is assumed to be in grayscale
    and the kernel is assumed to be square and with odd
    receptive field (size). A non-unit stride can be used,
    in which case a check on the feasibility of a complete
    tiling of the image is performed (in the negative case,
    an exception is raised).
    """
    # Check that the kernel size is odd.
    if kernel.shape[0] % 2 != 1:
        raise Exception('Kernel size must be odd')
    
    # Size of the "frame" around the center of the kernel.
    kernel_frame_size = int((kernel.shape[0] - 1) / 2)

    # Check that the image shape is compatible with the
    # kernel size and the stride.
    n_steps_width = 1. + (image.shape[0] - 2. * kernel_frame_size - 1.) / stride
    n_steps_height = 1. + (image.shape[1] - 2. * kernel_frame_size - 1.) / stride

    if not n_steps_width.is_integer():
        raise Exception('Image width, kernel receptive field and stride are incompatible')
    if not n_steps_height.is_integer():
        raise Exception('Image height, kernel receptive field and stride are incompatible')

    # Dimension of the output tensor.
    # output_dim = (
    #     image.shape[0] - 2 * kernel_frame_size,
    #     image.shape[1] - 2 * kernel_frame_size
    # )
    output_dim = (int(n_steps_width), int(n_steps_height))

    # Initialize the output tensor.
    output = tf.Variable(tf.zeros(shape=(output_dim[0], output_dim[1]), dtype=tf.float32))

    # Compute the convolution.
    # Loop over the rows of the image.
    for i in tqdm(range(output_dim[0])):
        # Loop over the columns of the image.
        for j in range(output_dim[1]):
            # Compute the coordinates of the center of the kernel.
            kernel_center_coords = [i * stride + kernel_frame_size, j * stride + kernel_frame_size]

            # Isolate the patch in the image overlapping with the kernel, given the
            # position of its center on the image itself.
            image_patch = image[
                kernel_center_coords[0] - kernel_frame_size: kernel_center_coords[0] + kernel_frame_size + 1,
                kernel_center_coords[1] - kernel_frame_size: kernel_center_coords[1] + kernel_frame_size + 1
            ]

            # Compute the element-by-element product of the image patch
            # with the kernel and take the sum of all the entries: that's
            # the (i, j) entry of the output tensor.
            output[i, j].assign(tf.reduce_sum(image_patch * kernel))
    
    return output

Load an image ("big" matrix) in grayscale (only one channel for each pixel).

In [ ]:
# A single grayscale image.
test_image = tf.cast(
    tf.random.uniform(minval=0, maxval=256, shape=(128, 128), dtype=tf.int32),
    dtype=tf.float32
)

test_image = tf.convert_to_tensor(tf.keras.utils.load_img(
    '../data/x-wing.jpeg',
    color_mode='grayscale',
    target_size=(200, 200),
    interpolation='nearest',
    keep_aspect_ratio=True
), dtype=tf.float32)

# Print the test image.
fig = plt.figure(figsize=(14, 6))

plt.imshow(
    test_image.numpy().astype('uint8'),
    cmap='gray'
)

In [ ]:
test_image.shape

Create kernels ("small" square matrices with odd size).

Blurring kernel.

In [ ]:
kernel_size = 11

# A kernel with all entries equal to 1/(kernel_size ** 2) should
# correspond to a blurring kernel.
blurring_kernel = tf.ones(shape=(kernel_size, kernel_size)) / (kernel_size ** 2)

# Compute convolution with stride 1.
print('Convolution with stride=1')

test_output = convolve(test_image, blurring_kernel)

# Print the resulting image.
fig = plt.figure(figsize=(14, 6))

plt.imshow(
    test_output.numpy().astype('uint8'),
    cmap='gray'
)


# Compute convolution with stride 3.
print('Convolution with stride=3')

test_output = convolve(test_image, blurring_kernel, stride=3)

# Print the resulting image.
fig = plt.figure(figsize=(14, 6))

plt.imshow(
    test_output.numpy().astype('uint8'),
    cmap='gray'
)


# Compute convolution with stride 7.
print('Convolution with stride=7')

test_output = convolve(test_image, blurring_kernel, stride=7)

# Print the resulting image.
fig = plt.figure(figsize=(14, 6))

plt.imshow(
    test_output.numpy().astype('uint8'),
    cmap='gray'
)

Sharpening kernel.

In [ ]:
sharpening_kernel = (
    - tf.linalg.band_part(tf.ones(shape=(5, 5)), 1, 1)
    + tf.linalg.tensor_diag([1., 1., 6., 1., 1.])
)

# Compute convolution with stride 1.
print('Convolution with stride=1')

sharpened_output = convolve(test_image, sharpening_kernel)

# Print the resulting image.
fig = plt.figure(figsize=(14, 6))

plt.imshow(
    sharpened_output.numpy().astype('uint8'),
    cmap='gray'
)


# Compute convolution with stride 3.
print('Convolution with stride=3')

sharpened_output = convolve(test_image, sharpening_kernel, stride=3)

# Print the resulting image.
fig = plt.figure(figsize=(14, 6))

plt.imshow(
    sharpened_output.numpy().astype('uint8'),
    cmap='gray'
)


# Compute convolution with stride 5.
print('Convolution with stride=5')

sharpened_output = convolve(test_image, sharpening_kernel, stride=5)

# Print the resulting image.
fig = plt.figure(figsize=(14, 6))

plt.imshow(
    sharpened_output.numpy().astype('uint8'),
    cmap='gray'
)